In [1]:
2+2

4

In [2]:
# https://partner.steamgames.com/doc/store/getreviews

In [26]:
import requests
import json
import csv
import sys
import os
import time  # Import time for the epoch timestamp
import re  # Import re for regex


# Construct the path to the scripts directory
script_path = os.path.abspath('../reviews-assistant/scripts')

# Add the path to sys.path
if script_path not in sys.path:
    sys.path.append(script_path)

import minsearch


class SteamReviewFetcher:
    def __init__(self, appids_with_titles, filter="all", language="english", day_range=30, review_type="all", purchase_type="all"):
        """
        Initializes the SteamReviewFetcher with the required parameters.

        :param appids_with_titles: List of tuples containing Steam application IDs and their corresponding titles.
        :param filter: Type of review filter.
        :param language: Language of the reviews.
        :param day_range: Number of days to consider for reviews.
        :param review_type: Type of review (all or specific).
        :param purchase_type: Type of purchase.
        """
        self.base_url = "https://store.steampowered.com/appreviews/"
        self.appids_with_titles = appids_with_titles if isinstance(appids_with_titles, list) else [appids_with_titles]
        self.filter = filter
        self.language = language
        self.day_range = day_range
        self.review_type = review_type
        self.purchase_type = purchase_type
        self.data_dir = os.path.abspath('../reviews-assistant/data/reviews')

        # Ensure the data directory exists
        os.makedirs(self.data_dir, exist_ok=True)

    def _construct_url(self, appid):
        return f"{self.base_url}{appid}?json=1"

    def _fetch_reviews(self, appid, num_reviews):
        url = self._construct_url(appid)
        params = {
            "filter": self.filter,
            "language": self.language,
            "day_range": self.day_range,
            "review_type": self.review_type,
            "purchase_type": self.purchase_type,
            "num_per_page": num_reviews
        }
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()

    def get_reviews(self, num_reviews=20, print_reviews=True):
        all_reviews = {}
        for appid, title in self.appids_with_titles:
            review_data = self._fetch_reviews(appid, num_reviews)
            reviews = review_data.get("reviews", [])
            all_reviews[appid] = (title, reviews)  # Store title along with reviews
            
            if print_reviews:
                self.print_first_last_reviews(appid, title, reviews)

        return all_reviews

    def print_first_last_reviews(self, appid, title, reviews):
        total_reviews = len(reviews)
        if total_reviews == 0:
            print(f"No reviews found for App ID {appid} ({title}).")
            return

        print(f"\nFirst 5 Reviews for App ID {appid} ({title}):")
        for review in reviews[:5]:
            self._print_review(review)

        print(f"\nLast 5 Reviews for App ID {appid} ({title}):")
        for review in reviews[-5:]:
            self._print_review(review)

    def _print_review(self, review):
        print(f"Author: {review['author']['steamid']}")
        print(f"Review: {review.get('review', 'No text')}")
        print(f"Rating: {'Positive' if review['voted_up'] else 'Negative'}")
        print(f"Timestamp: {review['timestamp_created']}")
        print("-" * 79)

    def _extract_columns_to_save(self, reviews, appid, title):
        extracted_reviews = []
        current_time = int(time.time())  # Get current epoch time
        for review in reviews:
            review_dict = {
                "appid": appid,  # Move appid to the first field
                "timestamp_query": current_time,  # Move timestamp_query to the second field
                "title": title,
            }
            for column in ["recommendationid", "author.steamid", "author.playtimeforever",
                           "author.playtime_last_two_weeks", "author.playtime_at_review",
                           "author.last_played", "language", "review", "timestamp_created", "timestamp_updated"]:
                if column.split('.')[0] == 'author':
                    nested_column = column.split('.')[1]
                    review_dict[column] = review['author'].get(nested_column)
                else:
                    review_dict[column] = review.get(column)
            extracted_reviews.append(review_dict)
        return extracted_reviews

    def save_reviews(self, all_reviews, filename_prefix, format):
        for appid, (title, reviews) in all_reviews.items():
            if not reviews:
                print(f"No reviews to save for App ID {appid} ({title}).")
                continue

            safe_title = self._sanitize_title(title)  # Sanitize title for filename
            lower_safe_title = safe_title.lower()  # Convert to lowercase for the filename
            if format == "csv":
                self._save_reviews_as_csv(appid, lower_safe_title, reviews, filename_prefix, title)
            elif format == "json":
                self._save_reviews_as_json(appid, lower_safe_title, reviews, filename_prefix, title)
            else:
                print("Invalid format. Please specify 'csv' or 'json'.")

    def _sanitize_title(self, title):
        """Remove special characters from the title for safe filename."""
        return re.sub(r'[<>:"/\\|?*]', '', title).replace("'", "").replace(" ", "_")

    def _save_reviews_as_csv(self, appid, lower_safe_title, reviews, filename_prefix, title):
        keys = ['appid', 'timestamp_query', 'title', 'steamid', 'review', 'voted_up', 'timestamp_created']  # Update the keys
        filename = os.path.join(self.data_dir, f"{filename_prefix}_{lower_safe_title}_{appid}_reviews.csv")
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(keys)  # Write the header
            for review in reviews:
                row = [
                    appid,  # App ID as the first field
                    int(time.time()),  # Current epoch timestamp as the second field
                    title,  # Keep the original title
                    review['author']['steamid'],
                    review.get('review', 'No text'),
                    'Positive' if review['voted_up'] else 'Negative',
                    review['timestamp_created'],
                ]
                writer.writerow(row)
        print(f"Reviews for App ID {appid} ({title}) saved to {filename}")

    def _save_reviews_as_json(self, appid, lower_safe_title, reviews, filename_prefix, title):
        filename = os.path.join(self.data_dir, f"reviews_{lower_safe_title}.json")
        extracted_reviews = self._extract_columns_to_save(reviews, appid, title)  # Use original title here
        with open(filename, 'w', encoding='utf-8') as jsonfile:
            json.dump(extracted_reviews, jsonfile, indent=4)
        print(f"Reviews for App ID {appid} ({title}) saved to {filename}")


# Example usage
if __name__ == "__main__":
    appids_with_titles = [
        ("2322010", "God of War: Ragnarok"),
        ("1086940", "Baldur's Gate 3")
    ]  # List of tuples containing app IDs and their titles
    review_fetcher = SteamReviewFetcher(appids_with_titles)

    num_reviews = 2000  # Specify the number of reviews to fetch
    print_reviews_flag = False  # Set to False to turn off printing reviews

    all_reviews = review_fetcher.get_reviews(num_reviews, print_reviews=print_reviews_flag)

    # Save reviews to file
    review_fetcher.save_reviews(all_reviews, "reviews", format="json")


Reviews for App ID 2322010 (God of War: Ragnarok) saved to /mnt/c/Users/KonuTech/llm-zoomcamp-capstone-01/reviews-assistant/data/reviews/reviews_god_of_war_ragnarok.json
Reviews for App ID 1086940 (Baldur's Gate 3) saved to /mnt/c/Users/KonuTech/llm-zoomcamp-capstone-01/reviews-assistant/data/reviews/reviews_baldurs_gate_3.json


# Ingestion

In [32]:
# Directory containing the data files
data_dir = os.path.abspath('../reviews-assistant/data/reviews')

# Initialize an empty list to hold all reviews
reviews = []

# List objects in the directory
objects_in_directory = os.listdir(data_dir)

# Iterate over the files in the directory
for obj in objects_in_directory:
    if obj.endswith('.json'):  # Check if the file is a JSON file
        file_path = os.path.join(data_dir, obj)
        with open(file_path, 'r', encoding='utf-8') as jsonfile:
            # Load the reviews from the JSON file
            file_reviews = json.load(jsonfile)
            reviews.extend(file_reviews)  # Append reviews to the main list
# Print the first i reviews
i = 2  # Change this to print more reviews if needed
for review in reviews[:i]:
    print(f"Author ID: {review['author.steamid']}")
    print(f"Review: {review.get('review', 'No text')}")
    print(f"Timestamp Created: {review['timestamp_created']}")
    print("-" * 79)

Author ID: 76561197996873645
Review: Awesome game, but I held off playing it because of the top-down/isometric camera - didn't like the camera in DOS2 either.  Then I found the "Native Camera Tweaks" mod on Nexus Mods.

This totally changes the game, allowing you to use standard 3rd person camera controls, and actually see what's ahead when moving up-hill.

It turns out all outdoor areas have sky boxes (including sun and moon), and almost all interior areas have ceilings.  Trust me, you'll hardly notice the few times a ceiling is missing or "pops in", unless you're looking for it.  And there's no performance hit that I've noticed.

Makes me wander why Larian didn't give an unlocked camera by default...

Also you have full control over zoom/pitch/FoV/invert axis/dead zones etc through the mod's configuration file.

[b][i]EDIT:  The mod has been updated (8 SEP 24) to work with patch 7.  Many thanks to Ersh for updating it so promptly!
Timestamp Created: 1724579126
-----------------------

In [34]:
index = minsearch.Index(
    # text_fields=["author.steamid", "author.playtimeforever", "author.playtime_last_two_weeks", "author.playtime_at_review", "author.last_played", "language", "review", "timestamp_created", "timestamp_updated"],
    text_fields=["title", "language", "review"],
    keyword_fields=["appid", "recommendationid"]
)

In [35]:
index.fit(reviews)

# RAG flow

In [36]:
# print(os.environ['OPENAI_API_KEY'])

In [37]:
from openai import OpenAI

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)


In [38]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [46]:
prompt_template = """
You're a conservative father of young children who is not aware of how the modern gaming industry works, who is not up to date with the titles released on a daily basis.
A father who needs to keep his child from not suitable games, including DEI games.
DEI ideology, which enforces Diversity, equity, and inclusion (DEI) hurts creativity, and uses corporate propaganda to improve sales.
Answer the QUESTION based on the CONTEXT from our exercises database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
title: {title}
language: {language}
review: {review}
""".strip()

def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [47]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [48]:
def rag(query, model='gpt-4o-mini'):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    #print(prompt)
    answer = llm(prompt, model=model)
    return answer

In [49]:
question = "Is Baldur's Gate 3 a game for kids?"
answer = rag(question)
print(answer)

Given the content of Baldur's Gate 3 and the reviews I've seen, it's important to consider whether it's suitable for young children. The game is an isometric turn-based RPG that involves complex themes, mature storytelling, character interactions, and choices that can significantly impact the narrative. 

Several reviews mention engaging with elements like dark magic, vampires, and morally ambiguous quests, which may not be appropriate for a younger audience. The game also features mature themes, including relationships and various character dynamics that may delve into adult concepts. 

Furthermore, the depth and complexity of the gameplay might be overwhelming for younger players unfamiliar with role-playing games or Dungeons & Dragons mechanics. 

Based on this information, it seems that Baldur's Gate 3 is likely not suitable for kids due to its mature content and complex themes. It may be more appropriate for older teens or adults who can fully grasp and appreciate its intricacies.

In [50]:
question = "Is Baldur's Gate 3 a game following DEI concepts, namely diversity, equity and inclusion?"
answer = rag(question)
print(answer)

Based on the available context regarding Baldur's Gate 3, there is no direct indication that the game is focused on or explicitly follows Diversity, Equity, and Inclusion (DEI) concepts. Reviews highlight the game's engaging storytelling, character depth, and immersive gameplay, but they do not mention any specific alignment with DEI ideology. Instead, the focus seems to be on the quality of the game design, character development, and the traditional aspects of Dungeons & Dragons.

While players can customize their characters, choose various races and classes, and experience impactful choices throughout the game, these elements are typical for RPGs and do not necessarily indicate an adherence to DEI principles. The overall emphasis in the reviews is on creativity, adventure, and personal experience rather than any corporate ideology related to DEI. 

Therefore, as a conservative father concerned with content, Baldur's Gate 3 does not appear to be a game that is structured around DEI co

In [51]:
question = "Is God of War Ragnarök a game for kids?"
answer = rag(question)
print(answer)

**God of War Ragnarök** is not suitable for young children. The game has been noted for containing themes that align with a "woke" ideology, which may not provide a wholesome experience for younger audiences. The narrative has been criticized for being overwhelmed with forced plotlines and character decisions, detracting from the overall experience. Moreover, the game involves mature themes, intense combat, and graphic content typical of the series, which may not be appropriate for kids. Therefore, as a conservative father looking to shield young children from unsuitable material, it would be wise to avoid this title for them.


In [52]:
question = "Is God of War Ragnarök a game following DEI concepts, namely diversity, equity and inclusion?"
answer = rag(question)
print(answer)

Yes, God of War Ragnarök exhibits elements that align with DEI concepts, particularly through its collaboration with a company known for inclusive storytelling. Critics have noted that this influence is apparent in certain plotlines and character decisions which may come across as forced, affecting the overall narrative experience. This "woke influence," as described in reviews, suggests that there are aspects of the game that lean into diversity, equity, and inclusion, which some players feel detracts from its creativity and storytelling. Therefore, if you are concerned about the suitability of this game for your children in relation to DEI ideology, you might want to consider these criticisms when making your decision.


# Retrieval evaluation

In [66]:
import os
import json

# Directory containing the ground truth file
data_dir = os.path.abspath('../reviews-assistant/data/ground_truth')

# Path to the ground_truth_retrieval.json file
file_path = os.path.join(data_dir, 'ground_truth_retrieval.json')

# Check if the file exists
if os.path.exists(file_path):
    try:
        # Open and load the JSON file
        with open(file_path, 'r', encoding='utf-8') as jsonfile:
            ground_truth_data = json.load(jsonfile)  # Load the JSON data into a Python object

        # Ensure each item is a dictionary and contains 'id' and 'question'
        if all(isinstance(item, dict) and 'id' in item and 'question' in item for item in ground_truth_data):
            print("Data successfully loaded and is in the correct format.")

            # Example: Print a sample of the data
            for item in ground_truth_data[:5]:  # Print the first 5 questions
                print(f"ID: {item['id']}, Question: {item['question']}")
        else:
            print("Error: The data format is incorrect or missing required fields ('id', 'question').")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
else:
    print(f"File {file_path} does not exist.")


Error: The data format is incorrect or missing required fields ('id', 'question').


In [67]:
ground_truth_data[0]

{'appid': '1086940',
 'question': "What kind of content does Baldur's Gate 3 have that I should be aware of as a father?"}

In [68]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [69]:
def minsearch_search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [70]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['appid']
        results = search_function(q)
        relevance = [d['appid'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [71]:
from tqdm.auto import tqdm

In [72]:
evaluate(ground_truth_data, lambda q: minsearch_search(q['question']))

  0%|          | 0/10 [00:00<?, ?it/s]

{'hit_rate': 1.0, 'mrr': 2.4323015873015867}

# Finding the best parameters

In [75]:
df_validation = ground_truth_data[:100]
df_test = ground_truth_data[100:]

In [78]:
df_validation

[{'appid': '1086940',
  'question': "What kind of content does Baldur's Gate 3 have that I should be aware of as a father?"},
 {'appid': '1086940',
  'question': 'Is this game suitable for children, or does it contain elements that are not appropriate for younger audiences?'},
 {'appid': '1086940',
  'question': "How does Baldur's Gate 3 portray its characters, and does it include any themes related to DEI ideology?"},
 {'appid': '1086940',
  'question': 'Can you explain why this game received high immersion ratings, and is that beneficial for young kids?'},
 {'appid': '1086940',
  'question': "Are there any potential negative messages or influences in Baldur's Gate 3 that I should consider before allowing my child to play?"},
 {'appid': '2322010',
  'question': 'Is God of War: Ragnarok suitable for young children, especially regarding its narrative and themes?'},
 {'appid': '2322010',
  'question': 'What are the main gameplay features of God of War: Ragnarok that might appeal to a fat

In [79]:
df_test

[]

In [76]:
import random

def simple_optimize(param_ranges, objective_function, n_iterations=10):
    best_params = None
    best_score = float('-inf')  # Assuming we're minimizing. Use float('-inf') if maximizing.

    for _ in range(n_iterations):
        # Generate random parameters
        current_params = {}
        for param, (min_val, max_val) in param_ranges.items():
            if isinstance(min_val, int) and isinstance(max_val, int):
                current_params[param] = random.randint(min_val, max_val)
            else:
                current_params[param] = random.uniform(min_val, max_val)
        
        # Evaluate the objective function
        current_score = objective_function(current_params)
        
        # Update best if current is better
        if current_score > best_score:  # Change to > if maximizing
            best_score = current_score
            best_params = current_params
    
    return best_params, best_score

In [80]:
def minsearch_search(query, boost=None):
    if boost is None:
        boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [83]:
param_ranges = {
    'exercise_name': (0.0, 3.0),
    'type_of_activity': (0.0, 3.0),
    'type_of_equipment': (0.0, 3.0),
    'body_part': (0.0, 3.0),
    'type': (0.0, 3.0),
    'muscle_groups_activated': (0.0, 3.0),
    'instructions': (0.0, 3.0),
}

def objective(boost_params):
    def search_function(q):
        return minsearch_search(q['question'], boost_params)

    results = evaluate(df_validation, search_function)
    return results['mrr']

In [84]:
simple_optimize(param_ranges, objective, n_iterations=20)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

({'exercise_name': 0.6619551952239617,
  'type_of_activity': 2.6401712720767545,
  'type_of_equipment': 2.834465828509363,
  'body_part': 1.7694674146572367,
  'type': 1.807127056545354,
  'muscle_groups_activated': 2.235120409175235,
  'instructions': 2.3535382107640945},
 2.4323015873015867)

In [86]:
def minsearch_improved(query):
    boost = {
        'exercise_name': 0.6619551952239617,
        'type_of_activity': 2.6401712720767545,
        'type_of_equipment': 2.834465828509363,
        'body_part': 1.7694674146572367,
        'type': 1.807127056545354,
        'muscle_groups_activated': 2.235120409175235,
        'instructions': 2.3535382107640945
    }

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

evaluate(ground_truth_data, lambda q: minsearch_improved(q['question']))

  0%|          | 0/10 [00:00<?, ?it/s]

{'hit_rate': 1.0, 'mrr': 2.4323015873015867}